In [16]:
#%pip install programy requests xmltodict

In [17]:
from ModelClasses import MainFunctions
from programy.clients.embed.datafile import EmbeddedDataFileBot

chatbot = EmbeddedDataFileBot(files={'aiml':['aiml_files']}, defaults=True)
apiKey = "5e533b50-76f9-11ec-81c5-e1cdfa92f93cc2df084c-c789-49ac-8839-eddcc0d946d1"
apiUrlRead = "https://machinelearningforkids.co.uk/api/scratch/"+ apiKey + "/classify"
apiUrlWrite = "https://machinelearningforkids.co.uk/api/scratch/"+ apiKey + "/train"
mainFunctions = MainFunctions(apiUrlRead, apiUrlWrite)
myClassifications = {
    "grafica_rendimiento": ["grafica_rendimiento"],
    "grafica_calidad": ["grafica_calidad"],
    "grafica_all": ["grafica_rendimiento", "grafica_calidad"],
    "procesador_rendimiento": ["procesador_rendimiento"],
    "procesador_calidad": ["procesador_calidad"],
    "procesador_all": ["procesador_rendimiento", "procesador_calidad"],
    "ram_rendimiento": ["ram_rendimiento"],
    "ram_calidad": ["ram_calidad"],
    "ram_all": ["ram_rendimiento", "ram_calidad"],
}

In [ ]:
ml4kTematicasCatalogo = sorted(set([item for sublist in myClassifications.values() for item in sublist]))

print("Saludos, soy un bot que te ayudará a encontrar los mejores componentes adaptados a tus necesidades")
print("Actualmente tenemos las siguientes temáticas en el catálogo:")

for tematica in ml4kTematicasCatalogo:
    print("->", mainFunctions.xmlToDict(chatbot.ask_question(tematica).rstrip("."))["descripcion"])

while True:
    print("-------------------------------------------------")
    texto = input("Dime en que estás interesado: ")
    ml4kAnswer = mainFunctions.classify(texto)
    
    # Eso no dice nada sobre grafica all, procesador all, ram all
    if(ml4kAnswer["confidence"] < 60):
        print("No he entendido lo que me quieres decir, pero puedo aprender de tí :)")
        print("Dime el número de la temática a la que te referías:")
        for i in range(len(ml4kTematicasCatalogo)):
            print("->", str(i+1) + ":", mainFunctions.xmlToDict(chatbot.ask_question(ml4kTematicasCatalogo[i]).rstrip("."))["descripcion"])
        print("->", str(len(ml4kTematicasCatalogo)+1) + ":", "No lo sé o descartar pregunta")
        indexTematica = int(input(""))-1
        if indexTematica < len(ml4kTematicasCatalogo):
            mainFunctions.store(texto, ml4kTematicasCatalogo[indexTematica])
    
    else:
        print("Estoy al", str(ml4kAnswer["confidence"]) + "%" ,"que quieres información sobre las siguientes categorías:")
        for categoria in myClassifications[ml4kAnswer["class_name"]]:
            print("->", mainFunctions.xmlToDict(chatbot.ask_question(categoria).rstrip("."))["descripcion"])
        arrCategoriasInteres = [mainFunctions.xmlToDict(chatbot.ask_question(x).rstrip(".")) for x in myClassifications[ml4kAnswer["class_name"]]]
        
        print("")
        for categoria in arrCategoriasInteres:
            print(categoria["texto"], ":")
            for index, componente in enumerate(categoria["componentes"]):
                print(str(index+1) + ".", componente)